In [6]:
# clear all the logs data
!rmdir logs_CIFAR10_Exam3_8

rmdir: failed to remove 'logs_CIFAR10_Exam3_8': Directory not empty


In [7]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import time

EPOCHS=200
NUM_CLASSES = 10
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.3
VERBOSE = 2
PATIENCE = 5
BOOL_TENSOR_BOARD = True

if BOOL_TENSOR_BOARD:
    %load_ext tensorboard

def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # normalize
    mean = np.mean(x_train,axis=(0,1,2,3)) # 평균
    std = np.std(x_train,axis=(0,1,2,3)) #표준편차
    x_train = (x_train-mean)/(std+1e-7) #batch Normalization
    x_test = (x_test-mean)/(std+1e-7) #batch Normalization
    y_train = tf.keras.utils.to_categorical(y_train,NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test,NUM_CLASSES)
    return x_train, y_train, x_test, y_test


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
def build_model():
    model = models.Sequential()
    # 1st block
    model.add(layers.Conv2D(32, (3,3), padding='same',
                            input_shape=x_train.shape[1:], 
                            activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
    
    # 2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
    
    # 3d block
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))
    
    # dense
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    return model

In [ ]:
(x_train, y_train, x_test, y_test) = load_data()
model = build_model()
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

start_training_time = time.time()

# train
if BOOL_TENSOR_BOARD:
    logdir = os.path.join("logs_CIFAR10_Exam3_8", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tb_callbacks = [
        #tf.keras.callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights = True, monitor='val_loss'),
                tf.keras.callbacks.TensorBoard(log_dir = logdir)]           
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,                     
                        callbacks = tb_callbacks, 
                        verbose = VERBOSE, validation_split =VALIDATION_SPLIT)
else:
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,                                             
                        verbose = VERBOSE, validation_split =VALIDATION_SPLIT)
    

end_training_time = time.time()

print("Training time = {0} minutes \n".format((end_training_time-start_training_time)/60))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)       

In [ ]:
trained_epochs = len(history.history['loss'])

print("Training and Validation Results")
print("Train [loss, accuracy] = [%0.4f, %0.4f]" % (history.history['loss'][trained_epochs-1], 
                                                   history.history['accuracy'][trained_epochs-1]))
print("Valid [loss, accuracy] = [%0.4f, %0.4f]" % (history.history['val_loss'][trained_epochs-1],
                                                   history.history['val_accuracy'][trained_epochs-1]))

In [ ]:

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'r-', label='loss')
plt.plot(history.history['val_loss'], 'b-', label='val_loss')
plt.xlabel('epoch')
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'r-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'b-', label='val_accuracy')
plt.xlabel('epoch')
plt.grid()
plt.legend(loc='lower right')
plt.show()


In [ ]:
# test
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

In [ ]:
a = 3; b = 5
print("{0} = {1}".format(a, b))